# Testing for E1 - using all spindles, training with all data
fistly define the UUID of the experiment to use

In [1]:
experimentId="37aac705-1a78-4277-9685-3e72d2ed14fe"

-------------------------------------------------------

In [2]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## load mass

In [3]:
samplerate=200  #Should rethink this

In [4]:
annotations, signalsMetadata = loadMASSSpindles(masspath,forceSamplerate=samplerate)

In [5]:
#consider to include this in a function
minDuration=0.3
maxDuration=5
annotations=annotations[annotations.type=='spindle']
annotations=annotations[annotations.duration>minDuration]
annotations=annotations[annotations.duration<maxDuration]
annotations=annotations.reset_index(drop=True)

In [6]:
#check with EDA results (not needed)
print(1-len(annotations)/33458)

0.005648873214178951


In [7]:
annotations.head()

,type,expert,subjectId,labelerId,startTime,duration,samplerate,stopTime,startInd,stopInd
0,spindle,E1,0001,0001,888.327805,0.640579,200,888.968384,177666,177794
1,spindle,E1,0001,0001,905.758061,0.578094,200,906.336155,181152,181267
2,spindle,E1,0001,0001,917.731574,0.847603,200,918.579177,183546,183716
3,spindle,E1,0001,0001,922.078189,0.878845,200,922.957034,184416,184591
4,spindle,E1,0001,0001,939.055445,0.757767,200,939.813212,187811,187963


In [8]:
signalsMetadata.head()

,subjectId,file,channel,duration,samplerate,isOriginalSamplerate,database
0,0001,MASS_0001.pkl,C3-CLE,28956.0,200,False,MASS
1,0002,MASS_0002.pkl,C3-CLE,35016.0,200,False,MASS
2,0003,MASS_0003.pkl,C3-CLE,36760.0,200,False,MASS
3,0004,MASS_0004.pkl,C3-CLE,28004.0,200,False,MASS
4,0005,MASS_0005.pkl,C3-CLE,31244.0,200,False,MASS


## Load experiment results

In [9]:
experimentModels, featureSelection = loadExperiment(experimentId,datapath)

In [10]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,aa11eb61-61e6-41e2-94d5-a4833e9b8d9d,0.040017
1,0001,E1,[0001],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,56daf263-9a04-44a8-a967-ed222c84ae8c,0.042790
2,0001,E1,[0001],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,983f3902-7ddc-4ab2-93e7-4da2577f8b4b,0.041762
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,c0eba1b0-cd60-4e9e-ab7f-71131a829c7e,0.037779
4,0001,E1,[0001],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,d7cf3254-9d32-4e20-a23e-3cdcd3107eaa,0.040051
5,0001,E1,[0001],"[0001, 0002, 0003, 0010, 0011, 0012, 0013, 001...","[0005, 0006, 0009]",0007,4f832318-c477-4835-acd3-7ec40944edd3,0.040469
6,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0007, 0010, 0011, 001...","[0002, 0017, 0013]",0009,15e578bc-6917-4cef-a16a-bade7c0b0676,0.037982
7,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0009, 0012, 001...","[0017, 0001, 0011]",0010,7e1fcff0-30f4-4f29-90d4-f7171b7700cf,0.037858
8,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0009, 0010, 0012, 001...","[0002, 0018, 0007]",0011,0f783e82-ff7a-4204-9e47-55e534671311,0.032807
9,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0009, 0011, 0013, 001...","[0001, 0007, 0010]",0012,99bfe384-9dde-485c-8233-eb9370c0bdbf,0.034677


In [11]:
#we show the difference in class inbalance for the annotation criteria considered
experimentModels[['criteriumName','spindleTimeRate']].groupby('criteriumName').describe()

spindleTimeRate                                          \
                        count      mean       std       min       25%   
criteriumName                                                           
E1                       15.0  0.038546  0.003350  0.032807  0.036428   
E2                       15.0  0.121634  0.007173  0.111565  0.116265   

                                             
                    50%       75%       max  
criteriumName                                
E1             0.037858  0.040260  0.045797  
E2             0.121738  0.126913  0.136630

In [12]:
featureSelection

,window,characteristic,bandName,score
0,2,hjortActivity,beta2,69.0
1,1,hjortActivity,sigma,63.0
2,2,hjortActivity,beta1,42.0
3,2,hjortActivity,delta2,41.0
4,1.5,sigmaIndex,broadband,39.0
5,2,sigmaIndex,broadband,39.0
6,2,hjortActivity,theta,38.0
7,2,hjortMobility,beta1,37.0
8,2,hjortActivity,alpha,35.0
9,1,sigmaIndex,broadband,34.0


## Hyperparameter definition
this should come from a previous evaluation notebook

In [13]:
hyperClose=0.1
hyperDuration=minDuration
hyperThres=0.3
hyperDepth=30

## Testing with E1 criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [14]:
experimentModels=experimentModels[experimentModels.criteriumName=='E1'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,aa11eb61-61e6-41e2-94d5-a4833e9b8d9d,0.040017
1,0001,E1,[0001],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,56daf263-9a04-44a8-a967-ed222c84ae8c,0.042790
2,0001,E1,[0001],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,983f3902-7ddc-4ab2-93e7-4da2577f8b4b,0.041762
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,c0eba1b0-cd60-4e9e-ab7f-71131a829c7e,0.037779
4,0001,E1,[0001],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,d7cf3254-9d32-4e20-a23e-3cdcd3107eaa,0.040051
5,0001,E1,[0001],"[0001, 0002, 0003, 0010, 0011, 0012, 0013, 001...","[0005, 0006, 0009]",0007,4f832318-c477-4835-acd3-7ec40944edd3,0.040469
6,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0007, 0010, 0011, 001...","[0002, 0017, 0013]",0009,15e578bc-6917-4cef-a16a-bade7c0b0676,0.037982
7,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0009, 0012, 001...","[0017, 0001, 0011]",0010,7e1fcff0-30f4-4f29-90d4-f7171b7700cf,0.037858
8,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0009, 0010, 0012, 001...","[0002, 0018, 0007]",0011,0f783e82-ff7a-4204-9e47-55e534671311,0.032807
9,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0009, 0011, 0013, 001...","[0001, 0007, 0010]",0012,99bfe384-9dde-485c-8233-eb9370c0bdbf,0.034677


In [15]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    testSubjectId=row.test
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres
    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    processedAnnotations=processedAnnotations[processedAnnotations.duration<maxDuration]
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)
    
    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels['rawF1']=rawF1s
experimentModels['rawPrecision']=rawPrecisions
experimentModels['rawRecall']=rawRecalls

experimentModels['f1']=f1s
experimentModels['precision']=precisions
experimentModels['recall']=recalls

experimentModels['eventF1']=eventF1s
experimentModels['eventPrecision']=eventPrecisions
experimentModels['eventRecall']=eventRecalls


*************************
1 of 15
*************************
2 of 15
*************************
3 of 15
*************************
4 of 15
*************************
5 of 15
*************************
6 of 15
*************************
7 of 15
*************************
8 of 15
*************************
9 of 15
*************************
10 of 15
*************************
11 of 15
*************************
12 of 15
*************************
13 of 15
*************************
14 of 15
*************************
15 of 15


In [16]:
dumpPickle('experimentModelsTest_E1_temp.pkl',experimentModels)

In [16]:
experimentModels.columns

Index(['criteriumId', 'criteriumName', 'labelerIdList', 'train', 'val', 'test',
       'modelId', 'spindleTimeRate', 'rawF1', 'rawPrecision', 'rawRecall',
       'f1', 'precision', 'recall', 'eventF1', 'eventPrecision',
       'eventRecall'],
      dtype='object')

In [17]:
fig=px.scatter(experimentModels,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [18]:
fig=px.scatter(experimentModels,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [19]:
fig=px.scatter(experimentModels,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [20]:
fig=px.scatter(experimentModels,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [21]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0001     1.0  0.706016 NaN  0.706016  0.706016  0.706016            1.0   
0002     1.0  0.779648 NaN  0.779648  0.779648  0.779648            1.0   
0003     1.0  0.618812 NaN  0.618812  0.618812  0.618812            1.0   
0005     1.0  0.622268 NaN  0.622268  0.622268  0.622268            1.0   
0006     1.0  0.664671 NaN  0.664671  0.664671  0.664671            1.0   
0007     1.0  0.619828 NaN  0.619828  0.619828  0.619828            1.0   
0009     1.0  0.783865 NaN  0.783865  0.783865  0.783865            1.0   
0010     1.0  0.735554 NaN  0.735554  0.735554  0.735554            1.0   
0011     1.0  0.639317 NaN  0.639317  0.639317  0.639317            1.0   
0012     1.0  0.683889 NaN  0.683889  0.683889  0.683889            1.0   
0013     1.0  0.673303 NaN  0.673303  0.673303  0.673303            1.0   
0014     1.0  0.593279 NaN  0.593279  0.593279  0.593279            1.0   
0017     1.0  0.736035 NaN  0.736035  0.736035  0.736035            1.0   
0018     1.0  0.733086 NaN  0.733086  0.733086  0.733086            1.0   
0019     1.0  0.537339 NaN  0.537339  0.537339  0.537339            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0001  0.866295 NaN  0.866295  0.866295  0.866295         1.0  0.595785 NaN   
0002  0.692726 NaN  0.692726  0.692726  0.692726         1.0  0.891514 NaN   
0003  0.478927 NaN  0.478927  0.478927  0.478927         1.0  0.874126 NaN   
0005  0.482258 NaN  0.482258  0.482258  0.482258         1.0  0.876833 NaN   
0006  0.603261 NaN  0.603261  0.603261  0.603261         1.0  0.740000 NaN   
0007  0.481212 NaN  0.481212  0.481212  0.481212         1.0  0.870614 NaN   
0009  0.768595 NaN  0.768595  0.768595  0.768595         1.0  0.799754 NaN   
0010  0.626998 NaN  0.626998  0.626998  0.626998         1.0  0.889308 NaN   
0011  0.480165 NaN  0.480165  0.480165  0.480165         1.0  0.957096 NaN   
0012  0.660105 NaN  0.660105  0.660105  0.660105         1.0  0.709450 NaN   
0013  0.522601 NaN  0.522601  0.522601  0.522601         1.0  0.945559 NaN   
0014  0.426352 NaN  0.426352  0.426352  0.426352         1.0  0.970506 NaN   
0017  0.758465 NaN  0.758465  0.758465  0.758465         1.0  0.714894 NaN   
0018  0.795775 NaN  0.795775  0.795775  0.795775         1.0  0.679553 NaN   
0019  0.368235 NaN  0.368235  0.368235  0.368235         1.0  0.993651 NaN   

                                    
           min       50%       max  
test                                
0001  0.595785  0.595785  0.595785  
0002  0.891514  0.891514  0.891514  
0003  0.874126  0.874126  0.874126  
0005  0.876833  0.876833  0.876833  
0006  0.740000  0.740000  0.740000  
0007  0.870614  0.870614  0.870614  
0009  0.799754  0.799754  0.799754  
0010  0.889308  0.889308  0.889308  
0011  0.957096  0.957096  0.957096  
0012  0.709450  0.709450  0.709450  
0013  0.945559  0.945559  0.945559  
0014  0.970506  0.970506  0.970506  
0017  0.714894  0.714894  0.714894  
0018  0.679553  0.679553  0.679553  
0019  0.993651  0.993651  0.993651

In [22]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0001,0.706016,0.866295,0.595785
1,0002,0.779648,0.692726,0.891514
2,0003,0.618812,0.478927,0.874126
3,0005,0.622268,0.482258,0.876833
4,0006,0.664671,0.603261,0.740000
5,0007,0.619828,0.481212,0.870614
6,0009,0.783865,0.768595,0.799754
7,0010,0.735554,0.626998,0.889308
8,0011,0.639317,0.480165,0.957096
9,0012,0.683889,0.660105,0.709450


In [23]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              6.668000e+54
eventF1           6.751273e-01
eventPrecision    6.007981e-01
eventRecall       8.339095e-01
dtype: float64

In [24]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_5308\2931376690.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.071015
eventPrecision    0.151791
eventRecall       0.120469
dtype: float64

In [25]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels.eventPrecision,
    'event F1':experimentModels.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels.eventRecall,
    'event F1':experimentModels.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [26]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

## Restrict to only N2 stage

In [27]:
experimentModels_N2=experimentModels.copy()
experimentModels_N2.head()

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate,rawF1,rawPrecision,rawRecall,f1,precision,recall,eventF1,eventPrecision,eventRecall
0,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,aa11eb61-61e6-41e2-94d5-a4833e9b8d9d,0.040017,0.645145,0.815861,0.533510,0.644936,0.818438,0.532129,0.706016,0.866295,0.595785
1,0001,E1,[0001],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,56daf263-9a04-44a8-a967-ed222c84ae8c,0.042790,0.722435,0.659882,0.798089,0.729392,0.668505,0.802481,0.779648,0.692726,0.891514
2,0001,E1,[0001],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,983f3902-7ddc-4ab2-93e7-4da2577f8b4b,0.041762,0.577951,0.447912,0.814387,0.591574,0.465971,0.809878,0.618812,0.478927,0.874126
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,c0eba1b0-cd60-4e9e-ab7f-71131a829c7e,0.037779,0.566200,0.435975,0.807356,0.583136,0.457833,0.802870,0.622268,0.482258,0.876833
4,0001,E1,[0001],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,d7cf3254-9d32-4e20-a23e-3cdcd3107eaa,0.040051,0.603478,0.568302,0.643295,0.613124,0.593872,0.633666,0.664671,0.603261,0.740000


In [28]:
annotationsN2=pd.read_csv(datapath+'/MASS/annotations/spindlesFilteredN2.csv')
annotationsN2['samplerate']=samplerate
annotationsN2['subjectId']=annotationsN2.apply(
    lambda row: str(row.subjectId).zfill(4),axis=1)
annotationsN2['labelerId']=annotationsN2.apply(
    lambda row: str(row.labelerId).zfill(4),axis=1)
annotationsN2['stopTime']=annotationsN2.apply(
    lambda row: row.startTime+row.duration , axis=1)
annotationsN2['startInd']=annotationsN2.apply(
    lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
annotationsN2['stopInd']=annotationsN2.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)

In [29]:
## load hypnograms
stagesAnnotations=pd.read_csv(datapath+"\MASS\stages\stages.csv")
stagesAnnotations['samplerate']=samplerate
stagesAnnotations['subjectId']=stagesAnnotations.apply(
    lambda row: str(row.subjectId).zfill(4),axis=1)
stagesAnnotations['stopTime']=stagesAnnotations.apply(
    lambda row: row.startTime+row.duration , axis=1)
stagesAnnotations['startInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
stagesAnnotations['stopInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)
stagesAnnotations.head(5)

hypnograms={}
for ind, row in signalsMetadata.iterrows():
    subjectId=row.subjectId
    thisStages=stagesAnnotations[stagesAnnotations.subjectId==subjectId]
    excerptDimension=int(row.duration*row.samplerate)
    thisHypnogram=np.ones((excerptDimension,))*np.nan
    for ind_stg, row_stg in thisStages.iterrows():
        thisHypnogram[row_stg.startInd:row_stg.stopInd]=row_stg.value
    hypnograms[subjectId]=thisHypnogram   

In [30]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels_N2.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels_N2)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    testSubjectId=row.test
    testHypnogram=hypnograms[testSubjectId]
    testN2Mask=testHypnogram==2
    #Define annotations criterium
    usedAnnotations=annotationsN2[annotationsN2.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres

    #Apply mask ---------------------------------->
    rawLabels=rawLabels*testN2Mask
    #<---------------------------------------------

    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    processedAnnotations=processedAnnotations[processedAnnotations.duration<maxDuration]
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)

    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels_N2['rawF1']=rawF1s
experimentModels_N2['rawPrecision']=rawPrecisions
experimentModels_N2['rawRecall']=rawRecalls

experimentModels_N2['f1']=f1s
experimentModels_N2['precision']=precisions
experimentModels_N2['recall']=recalls

experimentModels_N2['eventF1']=eventF1s
experimentModels_N2['eventPrecision']=eventPrecisions
experimentModels_N2['eventRecall']=eventRecalls


*************************
1 of 15
*************************
2 of 15
*************************
3 of 15
*************************
4 of 15
*************************
5 of 15
*************************
6 of 15
*************************
7 of 15
*************************
8 of 15
*************************
9 of 15
*************************
10 of 15
*************************
11 of 15
*************************
12 of 15
*************************
13 of 15
*************************
14 of 15
*************************
15 of 15


In [31]:
dumpPickle('experimentModelsTest_N2_E1_temp.pkl',experimentModels_N2)

In [32]:
fig=px.scatter(experimentModels_N2,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['rawF1'], y=experimentModels_N2['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [33]:
fig=px.scatter(experimentModels_N2,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['rawF1'], y=experimentModels_N2['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [34]:
fig=px.scatter(experimentModels_N2,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [35]:
fig=px.scatter(experimentModels_N2,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [36]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0001     1.0  0.728764 NaN  0.728764  0.728764  0.728764            1.0   
0002     1.0  0.809816 NaN  0.809816  0.809816  0.809816            1.0   
0003     1.0  0.708215 NaN  0.708215  0.708215  0.708215            1.0   
0005     1.0  0.660773 NaN  0.660773  0.660773  0.660773            1.0   
0006     1.0  0.720779 NaN  0.720779  0.720779  0.720779            1.0   
0007     1.0  0.724122 NaN  0.724122  0.724122  0.724122            1.0   
0009     1.0  0.815789 NaN  0.815789  0.815789  0.815789            1.0   
0010     1.0  0.830200 NaN  0.830200  0.830200  0.830200            1.0   
0011     1.0  0.689426 NaN  0.689426  0.689426  0.689426            1.0   
0012     1.0  0.770177 NaN  0.770177  0.770177  0.770177            1.0   
0013     1.0  0.714518 NaN  0.714518  0.714518  0.714518            1.0   
0014     1.0  0.625909 NaN  0.625909  0.625909  0.625909            1.0   
0017     1.0  0.776879 NaN  0.776879  0.776879  0.776879            1.0   
0018     1.0  0.758753 NaN  0.758753  0.758753  0.758753            1.0   
0019     1.0  0.535433 NaN  0.535433  0.535433  0.535433            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0001  0.938160 NaN  0.938160  0.938160  0.938160         1.0  0.595785 NaN   
0002  0.742129 NaN  0.742129  0.742129  0.742129         1.0  0.891089 NaN   
0003  0.595238 NaN  0.595238  0.595238  0.595238         1.0  0.874126 NaN   
0005  0.530142 NaN  0.530142  0.530142  0.530142         1.0  0.876833 NaN   
0006  0.702532 NaN  0.702532  0.702532  0.702532         1.0  0.740000 NaN   
0007  0.619828 NaN  0.619828  0.619828  0.619828         1.0  0.870614 NaN   
0009  0.832481 NaN  0.832481  0.832481  0.832481         1.0  0.799754 NaN   
0010  0.778390 NaN  0.778390  0.778390  0.778390         1.0  0.889308 NaN   
0011  0.538172 NaN  0.538172  0.538172  0.538172         1.0  0.959596 NaN   
0012  0.843434 NaN  0.843434  0.843434  0.843434         1.0  0.708628 NaN   
0013  0.574042 NaN  0.574042  0.574042  0.574042         1.0  0.945559 NaN   
0014  0.461022 NaN  0.461022  0.461022  0.461022         1.0  0.970506 NaN   
0017  0.850633 NaN  0.850633  0.850633  0.850633         1.0  0.714894 NaN   
0018  0.858849 NaN  0.858849  0.858849  0.858849         1.0  0.679553 NaN   
0019  0.366467 NaN  0.366467  0.366467  0.366467         1.0  0.993506 NaN   

                                    
           min       50%       max  
test                                
0001  0.595785  0.595785  0.595785  
0002  0.891089  0.891089  0.891089  
0003  0.874126  0.874126  0.874126  
0005  0.876833  0.876833  0.876833  
0006  0.740000  0.740000  0.740000  
0007  0.870614  0.870614  0.870614  
0009  0.799754  0.799754  0.799754  
0010  0.889308  0.889308  0.889308  
0011  0.959596  0.959596  0.959596  
0012  0.708628  0.708628  0.708628  
0013  0.945559  0.945559  0.945559  
0014  0.970506  0.970506  0.970506  
0017  0.714894  0.714894  0.714894  
0018  0.679553  0.679553  0.679553  
0019  0.993506  0.993506  0.993506

In [37]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0001,0.728764,0.938160,0.595785
1,0002,0.809816,0.742129,0.891089
2,0003,0.708215,0.595238,0.874126
3,0005,0.660773,0.530142,0.876833
4,0006,0.720779,0.702532,0.740000
5,0007,0.724122,0.619828,0.870614
6,0009,0.815789,0.832481,0.799754
7,0010,0.830200,0.778390,0.889308
8,0011,0.689426,0.538172,0.959596
9,0012,0.770177,0.843434,0.708628


In [38]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              6.668000e+54
eventF1           7.246369e-01
eventPrecision    6.821012e-01
eventRecall       8.339834e-01
dtype: float64

In [39]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_5308\813125388.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.077433
eventPrecision    0.169569
eventRecall       0.120686
dtype: float64

In [40]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels_N2.eventPrecision,
    'event F1':experimentModels_N2.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels_N2.eventRecall,
    'event F1':experimentModels_N2.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [41]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

### remove outliers

In [42]:
np.setdiff1d(experimentModels_N2.test,['0014','0019'])

array(['0001', '0002', '0003', '0005', '0006', '0007', '0009', '0010',
       '0011', '0012', '0013', '0017', '0018'], dtype=object)

In [43]:
experimentModels_N2[experimentModels_N2.test.isin(np.setdiff1d(experimentModels_N2.test,['0001','0014','0019']))][['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              1.666917e+43
eventF1           7.482873e-01
eventPrecision    7.054891e-01
eventRecall       8.291628e-01
dtype: float64

In [44]:
annotationPairToGraph(gtAnnotations,processedAnnotations)